In [95]:
import pandas as pd
import numpy as np

In [96]:
# Import data from CSV
df_asx = pd.read_csv('../data/ASX200_MONTHLY.csv')
df_zone_1a = pd.read_csv('../[3] - MACHINE_LEARNING/[3.1] - OUTLIER_REMOVAL/zone_data_1A_MONTHLY.csv')
df_zone_6 = pd.read_csv('../[3] - MACHINE_LEARNING/[3.1] - OUTLIER_REMOVAL/zone_data_6_MONTHLY.csv')
df_zone_7 = pd.read_csv('../[3] - MACHINE_LEARNING/[3.1] - OUTLIER_REMOVAL/zone_data_7_MONTHLY.csv')

format_asx = '%d/%m/%Y'  # e.g., '2023-01-31'
format_zone_1a = '%Y-%m-%d'  # e.g., '31/01/2023'
format_zone_6 = '%Y-%m-%d'
format_zone_7 = '%Y-%m-%d'  # e.g., '31-01-2023'


try:
    df_asx['newdate'] = pd.to_datetime(df_asx['Date'], format=format_asx).dt.to_period('M').dt.to_timestamp()
except ValueError:
    df_asx['newdate'] = pd.to_datetime(df_asx['Date'], errors='coerce').dt.to_period('M').dt.to_timestamp()

try:
    df_zone_1a['newdate'] = pd.to_datetime(df_zone_1a['date'], format=format_zone_1a).dt.to_period('M').dt.to_timestamp()
except ValueError:
    df_zone_1a['newdate'] = pd.to_datetime(df_zone_1a['date'], errors='coerce').dt.to_period('M').dt.to_timestamp()

try:
    df_zone_6['newdate'] = pd.to_datetime(df_zone_6['date'], format=format_zone_6).dt.to_period('M').dt.to_timestamp()
except ValueError:
    df_zone_6['newdate'] = pd.to_datetime(df_zone_6['date'], errors='coerce').dt.to_period('M').dt.to_timestamp()

try:
    df_zone_7['newdate'] = pd.to_datetime(df_zone_7['date'], format=format_zone_7).dt.to_period('M').dt.to_timestamp()
except ValueError:
    df_zone_7['newdate'] = pd.to_datetime(df_zone_7['date'], errors='coerce').dt.to_period('M').dt.to_timestamp()


# Set 'Date' as index and drop duplicates
df_asx.set_index('newdate', inplace=True, drop=True)
df_asx = df_asx[~df_asx.index.duplicated(keep='first')]

df_zone_1a.set_index('newdate', inplace=True, drop=True)
df_zone_1a = df_zone_1a[~df_zone_1a.index.duplicated(keep='first')]

df_zone_6.set_index('newdate', inplace=True, drop=True)
df_zone_6 = df_zone_6[~df_zone_6.index.duplicated(keep='first')]

df_zone_7.set_index('newdate', inplace=True, drop=True)
df_zone_7 = df_zone_7[~df_zone_7.index.duplicated(keep='first')]

# Create a common date index from the intersection of all DataFrames for the years 2023 to 2024
common_dates = df_asx.index.intersection(df_zone_1a.index).intersection(df_zone_6.index).intersection(df_zone_7.index)
common_dates = common_dates[(common_dates.year >= 2018) & (common_dates.year <= 2023)]

print(common_dates)

print("Zone 7 Dates:\n", df_zone_7.index)
print("ASX Dates:\n", df_asx.index)



DatetimeIndex(['2023-12-01', '2023-11-01', '2023-10-01', '2023-09-01',
               '2023-08-01', '2023-07-01', '2023-06-01', '2023-05-01',
               '2023-04-01', '2023-03-01',
               ...
               '2010-11-01', '2010-10-01', '2010-09-01', '2010-08-01',
               '2010-07-01', '2010-06-01', '2010-05-01', '2010-04-01',
               '2010-03-01', '2010-02-01'],
              dtype='datetime64[ns]', name='newdate', length=167, freq='-1MS')
Zone 7 Dates:
 DatetimeIndex(['2008-12-01', '2009-04-01', '2009-05-01', '2009-06-01',
               '2009-07-01', '2009-08-01', '2009-09-01', '2009-10-01',
               '2009-11-01', '2009-12-01',
               ...
               '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01',
               '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01',
               '2024-03-01', '2024-04-01'],
              dtype='datetime64[ns]', name='newdate', length=182, freq=None)
ASX Dates:
 DatetimeIndex(['2024-02-01', '2024-01

In [97]:

# Reindex to common dates
df_asx = df_asx.reindex(common_dates)
df_zone_1a = df_zone_1a.reindex(common_dates)
df_zone_6 = df_zone_6.reindex(common_dates)
df_zone_7 = df_zone_7.reindex(common_dates)


# Reset index to have 'Date' as a column again
df_asx.reset_index(inplace=True)
df_zone_1a.reset_index(inplace=True)
df_zone_6.reset_index(inplace=True)
df_zone_7.reset_index(inplace=True)



In [98]:



df_asx['Price'] = df_asx['Price'].str.replace(',', '').astype(float)
df_zone_1a['mean_price'] = pd.to_numeric(df_zone_1a['mean_price'], errors='coerce')
df_zone_6['mean_price'] = pd.to_numeric(df_zone_6['mean_price'], errors='coerce')
df_zone_7['mean_price'] = pd.to_numeric(df_zone_7['mean_price'], errors='coerce')


df_asx = df_asx.iloc[::-1]
df_zone_1a = df_zone_1a.iloc[::-1]
df_zone_6 = df_zone_6.iloc[::-1]
df_zone_7 = df_zone_7.iloc[::-1]

print(df_zone_1a)

# Compute returns for each DataFrame
returns_asx = df_asx['Price'].pct_change().dropna()
returns_zone_1a = df_zone_1a['mean_price'].pct_change().dropna()
returns_zone_6 = df_zone_6['mean_price'].pct_change().dropna()
returns_zone_7 = df_zone_7['mean_price'].pct_change().dropna()

print("ASX Returns:\n", returns_asx)
print("Zone 1A Returns:\n", returns_zone_1a)
print("Zone 6 Returns:\n", returns_zone_6)
print("Zone 7 Returns:\n", returns_zone_7)


       newdate  Unnamed: 0        date   mean_price  IQR_price  mean_volume  \
166 2010-02-01          10  2010-02-28  2067.162407   575.0000    79.537037   
165 2010-03-01          11  2010-03-31  1893.861765   222.5000    37.588235   
164 2010-04-01          12  2010-04-30  2064.062500   225.0000    62.184375   
163 2010-05-01          13  2010-05-31  2021.354286   189.7225    61.133333   
162 2010-06-01          14  2010-06-30  1983.308841   550.0000    74.515942   
..         ...         ...         ...          ...        ...          ...   
4   2023-08-01         172  2023-08-31  4090.303030   600.0000    21.063636   
3   2023-09-01         173  2023-09-30  4000.892857   230.0000    40.108929   
2   2023-10-01         174  2023-10-31  3984.351940   174.8050    33.135821   
1   2023-11-01         175  2023-11-30  4026.571429   150.0000    46.960000   
0   2023-12-01         176  2023-12-31  4006.749500   100.0000    64.967500   

     IQR_volume  
166      83.700  
165      27.750

In [99]:

# Combine returns into a single DataFrame if needed
returns_combined = pd.concat([returns_asx, returns_zone_1a, returns_zone_6, returns_zone_7], axis=1)
returns_combined.columns = ['ASX', 'Zone_1A', 'Zone_6', 'Zone_7']




# Combine returns into a single DataFrame if needed
returns_combined = pd.concat([returns_asx, returns_zone_1a, returns_zone_6, returns_zone_7], axis=1)
returns_combined.columns = ['ASX', 'Zone_1A', 'Zone_6', 'Zone_7']





# Compute covariance matrix for the combined returns
covariance_matrix = returns_combined.cov()

# Display the results
print("Returns:\n", returns_combined)
print("\nCovariance Matrix:\n", covariance_matrix)

Returns:
           ASX   Zone_1A    Zone_6    Zone_7
165  0.051275 -0.083835 -0.085465  0.033682
164 -0.013968  0.089870  0.043524 -0.001192
163 -0.078566 -0.020691 -0.155888 -0.058681
162 -0.028941 -0.018822  0.253868  0.054909
161  0.044636  0.051777 -0.019569 -0.047555
..        ...       ...       ...       ...
4   -0.014183  0.022133 -0.053892 -0.014284
3   -0.035139 -0.021859 -0.005425 -0.024485
2   -0.038008 -0.004134 -0.003030 -0.012739
1    0.045217  0.010596 -0.002660 -0.016447
0    0.071043 -0.004923 -0.008711  0.025492

[166 rows x 4 columns]

Covariance Matrix:
               ASX   Zone_1A    Zone_6    Zone_7
ASX      0.001585 -0.000139 -0.000081 -0.000161
Zone_1A -0.000139  0.001817  0.000356  0.000574
Zone_6  -0.000081  0.000356  0.004357  0.000457
Zone_7  -0.000161  0.000574  0.000457  0.001749


In [100]:
returns_combined.to_csv('COMBINED_RETURNS.csv', index=False)
print("Combined returns data saved to 'combined_returns.csv'")


Combined returns data saved to 'combined_returns.csv'
